# Process Inputs:

## Imports

In [1]:
import os
import scipy
import numpy as np
from scipy.io import wavfile
import pandas as pd
from scipy.signal import spectrogram, get_window
from os import listdir

## stft

In the stft function we find the normalized frequency values for each frequency bin and time bin for a given data d.

We also define the window sizes for the stft.

In [2]:
def stft (d,fs,winlen,nfft):
    noverlap=fs/100
    nperseg=winlen
    win = get_window('hamming',winlen)
    freqbin,tbin,spec=spectrogram(x=d,fs=fs,noverlap=noverlap,nperseg=nperseg,nfft=nfft,window=win)
    spec=(spec*1000)+1
    spec=np.real(np.log(spec))
    spec[spec<0]=0
    return spec, freqbin, tbin

In [3]:
winlen1=2048
nfft1=2048

winlen2=8192
nfft2=8192

## genData

In the genData function we find the frequency values of each note frequency bin and time bin for a given spectrogram spec.

To do this we find all those frequency bins that lie in the range between two note frequencies. If there are multiple bins in this range, we find a weighted average by finding the bin closest to the actual note frequency and assigning weights based on 2 stright lines, acending from the first frequency bin in the range, reaching 1 at the closest bin to the actual note, and decending until the last frequency bin.

We also compute the differences between each time bin and it's previous frequecny, assigning 0s to the initial time bin.

In [4]:
def genData (spec,freqbin,tbin):
    Q=np.zeros([tbin.shape[0],Notes.shape[0]])
    for i in range(spec.shape[1]):
        Z=spec[:,i]
        for j in range(Notes.shape[0]-1):
            if j==0:
                j=1
            f1=Notes[j-1]
            f2=Notes[j+1]
            wb1 = np.where(f1>=freqbin)[0]
            wb2 = np.where(f2<=freqbin)[0]
            wb=np.intersect1d(wb1,wb2)
            bn=np.argmin(np.abs(freqbin-Notes[j]))
        
            if ((wb.shape[0]==0) or wb.shape[0]==1):
                Q[i,j]=Z[j]
            
            else:
                x1 = wb[0:np.where(wb==bn)[0][0]+1]
                x2 = wb[np.where(wb==bn)[0][0]:]
                y1 = (x1-wb[0])/(x1.shape[0]-1)
                y2 = 1-(x2-bn)/(x2.shape[0]-1)
                y1[np.isnan(y1)]=1
                y2[np.isnan(y2)]=1
                Q[i,j]= ((np.matmul(Z[x1],y1)+np.matmul(Z[x2],y2)-Z[bn])/(np.sum(y1)+np.sum(y2)))
    Q=Q[:,1:101]
    b=np.zeros((1,Q.shape[1]))
    a=np.diff(Q,axis=0)
    a=np.concatenate((b,a),axis=0)
    Q=np.concatenate((Q,a),axis=1)
    return Q

## Notes

Here we define the frequencies of notes starting at note 0 - frequency 0 and ending at note 101 -  frequecny 8870

In [5]:
# Define frequencies for notes up to 100 (first harmonic of 88th note)
Notes = np.zeros(102)
Notes[0] = 0
Notes[1] = 27.5
for i in range(100):
    Notes[i+2] = Notes[i+1]*(np.power(2,1/12))

## Process

Process data and write to file

In [87]:
dir_path_in = os.path.join(os.getcwd(),'Data/Recordings/Testing')
out_path = os.path.join(os.getcwd(),'Data/Processed/Testing/input.txt')
out_path1 = os.path.join(os.getcwd(),'Data/Processed/Testing/tbin.txt')

files = listdir(dir_path_in)
inp_final = np.zeros((1,400))
out_final = np.zeros((1,88))
for i in range(len(files)):
    out_path = os.path.join(os.getcwd(),'Data/Processed/Testing/input/'+files[i][:-4]+'.txt')
    out_path1 = os.path.join(os.getcwd(),'Data/Processed/Testing/tbin/'+files[i][:-4]+'.txt')
    wav_path = os.path.join(os.getcwd(), 'Data/Recordings/Testing', files[i])
    fs, data = wavfile.read(wav_path)
    data = data[:,0]
    spec1,freqbin1,tbin1 = stft(d=data,fs=fs,winlen=winlen1,nfft=nfft1)
    spec2,freqbin2,tbin2 = stft(d=data,fs=fs,winlen=winlen2,nfft=nfft2)
    
    
    Q1_temp = genData(spec=spec1,freqbin=freqbin1,tbin=tbin1)
    Q2 = genData(spec=spec2,freqbin=freqbin2,tbin=tbin2)

    Q1 = Q2.copy()
    
    
    for j in range(tbin2.shape[0]):
        near_t = np.argmin(np.abs(tbin2[j]-tbin1))
        Q1[j,:]=Q1_temp[near_t,:]
    
    inp_final = np.append(inp_final,np.concatenate((Q1,Q2),axis=1),axis=0)
    
    np.savetxt(out_path1,tbin2,delimiter=',')
    np.savetxt(out_path,inp_final,delimiter=',')
    
    print('Done processing',files[i])

Done processing Viper's Drag.wav
Done processing After You've Gone.wav
Done processing African Ripples.wav
Done processing Ain't Misbehavin'.wav
Done processing Begin the Beguine.wav
Done processing Blue And Sentimental.wav
Done processing Easy Living.wav
Done processing Fur Elise.wav
Done processing It's Always You.wav
Done processing MozartK265.wav
Done processing Scale.wav
Done processing Stardust.wav


'Stardust'